In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

# 크롬 드라이버 자동 업데이트
chrome_driver_path = ChromeDriverManager().install()

# Selenium 웹 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저를 띄우지 않고 백그라운드에서 실행
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 기존의 real_estate.pdf 파일 삭제
pdf_file_path = "researches/pdfs/real_estate.pdf"
if os.path.exists(pdf_file_path):
    os.remove(pdf_file_path)
    #print("기존의 real_estate.pdf 파일 삭제")

# PDF 파일 생성
c = canvas.Canvas(pdf_file_path, pagesize=letter)

# 한글 폰트 설정
font_path = "./font/NanumGothic.ttf"  # 폰트 파일 경로를 설정해주세요
pdfmetrics.registerFont(TTFont("Korean", font_path))

# 1페이지부터 5페이지까지 반복
for page_num in range(1, 6):
    # 1. https://omoney.kbstar.com/quics?page=C042015&cc=b037806:b037806에 진입한다.
    url = f"https://omoney.kbstar.com/quics?page=C042015&cc=b037806:b037806&page={page_num}"
    driver.get(url)
    
    # 2. html에서 class="tbl_list"안에서 href 목록과 제목을 구하는 코드
    tbl_list = driver.find_element(By.CLASS_NAME, "tbl_list")
    links = tbl_list.find_elements(By.TAG_NAME, "a")
    titles = [link.text for link in links]
    hrefs = [link.get_attribute("href") for link in links]

    # 3. 각 href와 제목를 PDF 페이지로 추가
    for title, href in zip(titles, hrefs):
        driver.get(href)
        time.sleep(1)  # 페이지 로딩 대기
        view_cont = driver.find_element(By.ID, "view_cont").text
        
        # 줄바꿈을 처리하고, 빈 줄인 경우는 제외
        lines = view_cont.split("\n")
        lines = [line for line in lines if line.strip() != '']
        
        # 페이지별로 PDF 페이지에 추가
        c.setFont("Korean", 8)  # 폰트 크기를 8로 설정
        c.drawString(100, 700, f"Title: {title}")  # 제목 추가
        y_position = 690  # 내용 시작 위치
        for line in lines:
            c.drawString(100, y_position, line)  # 내용 추가
            y_position -= 10  # 줄 간격 조절
        
        c.showPage()  # 페이지 추가

driver.quit()
c.save()  # PDF 파일 저장
#print("real_estate.pdf 파일 생성 완료")
